# Porous Electrode FiPy model

Troubleshooting of model presented in [usnistgov/fipy#788](https://github.com/usnistgov/fipy/issues/788)

In [1]:
%matplotlib inline

$ \frac{\partial c_1}{\partial t} = -\nabla N_1 - \nabla i/F $
$ \frac{\partial c_2}{\partial t} = -\nabla N_2 + \nabla i/F $
$ \sum{z_i \nabla N_i} = \nabla i/F $
where
$ N_i = -D*\nabla c_i - Dz_ic_i*\nabla \phi_2 $
and
$ \nabla i = ai0/por(c_1/c_1^0 e^{z_1ctF/(RT)(\phi_1 - \phi_2 - \phi_0)} - c_2/c_2^0 e^{z_2ctF/(RT)(\phi_1 - \phi_2 - \phi_0)}) $

a,i0,F,D,phi_1,z_1,ct,R,T, cm_1^0, and c_2^0 are constants
phi_2, c_1, and c_2 are time-dependent, time-evolving variables to solve for. In the problem, z_2 is 0 which simplifies the flux and exponent terms for c_2.

In implementing the flux terms in my code given by N, the chain rule is used to better couple the system and try to stay implicit:

$\nabla (c \nabla \phi) \equiv \nabla c \cdot \nabla \phi + c \nabla^2 \phi$

For simplicity, start with 0 flux boundary conditions on the bounds of x on [0,L].
The initial conditions are:

$ c_1(x,0) = 0.1 $
$ c_2(x,0) = 0. $
$ \phi_2 (x,0) = \phi_0 $

When I run the attached code, I get "RuntimeError: Factor is exactly singular" I've messed around with this code a bit, and in some formulations of this problem, with very small time changes, the code will solve, but the values don't change. I've looked into changing the iterations or tolerance of the solver, but it either doesn't change the final solution or the code doesn't run. Another issue is the long runtime because dt seems limited by the diffusion coefficient. Here's the code. Any help is greatly appreciated, as I've tried making my own solver but found that FiPy can greatly reduce my number of lines of code so it may be easier to de-bug.

In [ ]:
import numpy as np
import scipy
from fipy import *
from fipy import Variable, FaceVariable, CellVariable, Grid1D, TransientTerm, DiffusionTerm, Viewer
from fipy import ImplicitSourceTerm, ExplicitDiffusionTerm
from fipy.tools import numerix
from builtins import range

phi_0 = 0.0 # arbitrarily set to 0
phi_app = 0.2 # Applied voltage 
c1_0 = 0.1 # M  
c2_0 = 0.1 # M
z1 = -1.0
z2 = 0.
phi_1 = phi_0 + phi_app/2 
L = .25 # cm of electrode thickness 
por = 0.4 # porosoity 
D = 5*10**-5 # diffusion coefficient, cm^2/s
a = 360. #cm^2 per cm^3, specific interfacial area for GFD2.5
ct,R,F,T = 0.5, 8.3143, 96487., 293.
ne, s = 1, 1
i0 = 0.005 # from Exchange Current Densities 

nx = 10000
dx = L/nx
mesh = Grid1D(nx=nx,dx=dx)
x = mesh.cellCenters[0]
#dt = .01
dt = 0.5*dx**2/(2*D)
steps = 1000
t = dt * steps
time = Variable(0.)

phi2 = CellVariable(name="phi2",mesh=mesh,hasOld=True)
c1 =  CellVariable(name="c1",mesh=mesh,hasOld=True)
c2 =  CellVariable(name="c2",mesh=mesh,hasOld=True)
phi2.setValue(phi_0)
c1.setValue(c1_0)
c2.setValue(c2_0)

# No flux at one end
c1.faceGrad.constrain(0,mesh.facesRight)
c2.faceGrad.constrain(0,mesh.facesRight)
phi2.faceGrad.constrain(0,mesh.facesRight)

# No flux at other end
c1.faceGrad.constrain(0,mesh.facesLeft)
c2.faceGrad.constrain(0,mesh.facesLeft)
phi2.faceGrad.constrain(0,mesh.facesLeft)


far = a*i0/F*(c1/c1_0*numerix.exp(z1*ct*F*(phi_1-phi2-phi_0)/(R*T))-\
              c2/c2_0*numerix.exp(-z2*ct*F*(phi_1-phi2-phi_0)/(R*T)))

# Equation for first species
eq1 = TransientTerm(var=c1) == DiffusionTerm(coeff=D,var=c1) \
# add migration
+ DiffusionTerm(coeff=c1*z1*D,var=phi2)
# Replaced below  line w/ above because below is not
# very implicit and we want to couple the equations
+ (ImplicitSourceTerm(coeff=phi2.faceGrad.divergence*D*z1,var=c1) 
- phi2.grad.dot(c1.grad)*D*z1)  \

# subtract Faradaic term
- a*i0/F/por*c1/c1_0*numerix.exp(z1*ct*F*(phi_1-phi2-phi_0)/(R*T)) \
- ImplicitSourceTerm(coeff=a*i0*(-c2/c2_0),var=None) 
# Used above line because z2=0

# c2 equation; migration has no effect, z2=0. 
# Faradaic term is added, broken into
# two terms, one to be a variable of c2
eq2 = TransientTerm(var=c2) == DiffusionTerm(coeff=D,var=c2) \
+ ImplicitSourceTerm(coeff=-a*i0/F/por/c2_0,var=c2) 
# z2=0 for ^^
+ ImplicitSourceTerm(a*i0/por/F*(c1/c1_0*numerix.exp(z1*ct*F*(phi_1- \
                                                              phi2 - phi_0)/(R*T))),var=None)

# Equation relating flux to current for electrolyte potential
# Can't simplify to one Diffusion term because 
# of z1 multiplying both flux terms
eq3 = DiffusionTerm(coeff=c1*D,var=phi2) +  D*c1.grad.dot(phi2.grad) \
    # Add diffusion
# + D*z1*c1.faceGrad.divergence <--- don/t use this?
+ DiffusionTerm(coeff=z1*D,var=c1)
# Add Faradaic contributions, 
# can't use ImplicitSourceTerm because it's nonlinear in phi
+ a*i0/F/por*(c1/c1_0*numerix.exp(z1*ct*F*(phi_1-phi2-phi_0)/(R*T)) \
              -c2/c2_0*numerix.exp(-z2*ct*F*(phi_1-phi2-phi_0)/(R*T)))


eq = eq3&eq1&eq2

# mySolver = LinearPCGSolver(iterations=3000,tolerance=1e-10)
from builtins import range
result_c1 = []
result_c2 = []
result_phi2 = []
t_list = []
for i in range(steps):
    t_list.append(float(i*dt))
    result_c1.append(float(c1.max()))
    result_c2.append(float(c2.max()))
    result_phi2.append(float(phi2.max()))
    c1.updateOld()
    c2.updateOld()
    phi2.updateOld()
    for j in range(5):
        res=eq.sweep(dt=dt)
        print(res)
    #eq1.solve(c1,dt=dt)
    #eq2.solve(c2,dt=dt)
    #eq3.solve(var=phi2,solver=mySolver,dt=dt)
    print(i,'step')
    viewer = Matplotlib1DViewer(vars=(c1,c2,phi2))
    viewer.plot()